# Project 3: **Ask‑the‑Web Agent**

Welcome to Project 3! In this project, you will learn how to use tool‑calling LLMs, extend them with custom tools, and build a simplified *Perplexity‑style* agent that answers questions by searching the web.

## Learning Objectives  
* Understand why tool calling is useful and how LLMs can invoke external tools.
* Implement a minimal loop that parses the LLM's output and executes a Python function.
* See how *function schemas* (docstrings and type hints) let us scale to many tools.
* Use **LangChain** to get function‑calling capability for free (ReAct reasoning, memory, multi‑step planning).
* Combine LLM with a web‑search tool to build a simple ask‑the‑web agent.

## Roadmap
1. Environment setup
2. Write simple tools and connect them to an LLM
3. Standardize tool calling by writing `to_schema`
4. Use LangChain to augment an LLM with your tools
5. Build a Perplexity‑style web‑search agent
6. (Optional) A minimal backend and frontend UI

# 1- Environment setup

## 1.1- Conda environment

Before we start coding, you need a reproducible setup. Open a terminal in the same directory as this notebook and run:

```bash
# Create and activate the conda environment
conda env create -f environment.yml && conda activate web_agent

# Register this environment as a Jupyter kernel
python -m ipykernel install --user --name=web_agent --display-name "web_agent"
```
Once this is done, you can select “web_agent” from the Kernel → Change Kernel menu in Jupyter or VS Code.


> Behind the scenes:
> * Conda reads `environment.yml`, resolves the pinned dependencies, creates an isolated environment named `web_agent`, and activates it.
> * `ollama pull` downloads the model so you can run it locally without API calls.


## 1.2 Ollama setup

In this project, we start with `gemma3-1B` because it is lightweight and runs on most machines. You can try other smaller or larger LLMs such as `mistral:7b`, `phi3:mini`, or `llama3.2:1b` to compare performance. Explore available models here: https://ollama.com/library

```bash
ollama pull llama3.2:3b
```

`ollama pull` downloads the model so you can run it locally without API calls.


## 2- Tool Calling

LLMs are strong at answering questions, but they cannot directly access external data such as live web results, APIs, or computations. In real applications, agents rarely rely only on their internal knowledge. They need to query APIs, retrieve data, or perform calculations to stay accurate and useful. Tool calling bridges this gap by allowing the LLM to request actions from the outside world.


We describe each tool’s interface in the model’s prompt, defining what it does and what arguments it expects. When the model decides that a tool is needed, it emits a structured output like: `TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Francisco"}}`. Your code will detect this output, execute the corresponding function, and feed the result back to the LLM so the conversation continues.

In this section, you will implement a simple `get_current_weather` function and teach the `gemma3` model how to use it when required in four steps:
1. Implement the tool
2. Create the instructions for the LLM
3. Call the LLM with the prompt
4. Parse the LLM output and call the tool

In [41]:
# ---------------------------------------------------------
# Step 1: Implement the tool
# ---------------------------------------------------------
# Your goal: give the model a way to access weather information.
# You can either:
#   (a) Call a real weather API (for example, OpenWeatherMap), or
#   (b) Create a dummy function that returns a fixed response (e.g., "It is 23°C and sunny in San Francisco.")
#
# Requirements:
#   • The function should be named `get_current_weather`
#   • It should take two arguments:
#         - city: str
#         - unit: str = "celsius"
#   • Return a short, human-readable sentence describing the weather.
#
# Example expected behavior:
#   get_current_weather("San Francisco") → "It is 23°C and sunny in San Francisco."
#


def get_current_weather_dummy(city: str, unit: str = "celsius") -> str:
    temperature = "23°C" if unit == "celsius" else "73°F"
    return f"It is {temperature} and sunny in {city}."


import requests

API_KEY = "2733b92981329a239d31a756bb055903"  # Replace with your actual key

def get_current_weather(city: str, unit: str = "celsius") -> str:
    # Map unit to OpenWeatherMap format
    units_map = {"celsius": "metric", "fahrenheit": "imperial"}
    units = units_map.get(unit.lower(), "metric")

    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units={units}"
    
    try:
        response = requests.get(url, timeout=10)
        
        if response.status_code != 200:
            # Show more detailed error info
            try:
                error_data = response.json()
                error_msg = error_data.get('message', 'Unknown error')
                return f"Could not retrieve weather for {city}. API Error ({response.status_code}): {error_msg}"
            except:
                return f"Could not retrieve weather for {city}. Status code: {response.status_code}"
        
        data = response.json()
        temp = data["main"]["temp"]
        description = data["weather"][0]["description"]
        return f"It is {temp}°{'C' if units=='metric' else 'F'} and {description} in {city}."
    
    except requests.exceptions.Timeout:
        return f"Request timed out while getting weather for {city}."
    except requests.exceptions.RequestException as e:
        return f"Network error getting weather for {city}: {str(e)}"


In [69]:
# Test the weather API directly to see the exact error
print("Testing weather API directly...")
result = get_current_weather("Seattle,US", unit="celsius")
print(f"Result: {result}")

# Also test the URL directly
url = f"http://api.openweathermap.org/data/2.5/weather?q=Seattle&appid={API_KEY}&units=metric"
print(f"\nTesting URL: {url}")
response = requests.get(url)
print(f"Status Code: {response.status_code}")
if response.status_code != 200:
    print(f"Error Response: {response.text}")

Testing weather API directly...
Result: It is 6.36°C and light rain in Seattle,US.

Testing URL: http://api.openweathermap.org/data/2.5/weather?q=Seattle&appid=2733b92981329a239d31a756bb055903&units=metric
Status Code: 200


In [44]:
# ---------------------------------------------------------
# Step 2: Create the prompt for the LLM to call tools
# ---------------------------------------------------------
# Goal:
#   Build the system and user prompts that instruct the model when and how
#   to use your tool (`get_current_weather`).
#
# What to include:
#   • A SYSTEM_PROMPT that tells the model about the tool use and describe the tool
#   • A USER_QUESTION with a user query that should trigger the tool.
#       Example: "What is the weather in San Diego today?"

# Try experimenting with different system and user prompts
# ---------------------------------------------------------


# ---------------------------------------------------------
# Step 2: Create the prompt for the LLM to call tools
# ---------------------------------------------------------

SYSTEM_PROMPT = """
You are an assistant that can answer questions and call tools when needed.

Available tool:
- get_current_weather(city: str, unit: str = "celsius") → Returns a short sentence describing the weather.

When you need to use a tool, output ONLY in this format:
TOOL_CALL: {"name": "<tool_name>", "args": {"city": "<city>", "unit": "<unit>"}}

Example:
TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Francisco", "unit": "celsius"}}

Do NOT include any extra text outside this JSON format when calling a toolDo NOT include any extra text outside this JSON format when calling a tool.
"""


In [45]:
# ---------------------------------------------------------
# Step 2: Create the prompt for the LLM to call tools
# ---------------------------------------------------------
# Goal:
#   Build the system and user prompts that instruct the model when and how
#   to use your tool (`get_current_weather`).
#
# What to include:
#   • A SYSTEM_PROMPT that tells the model about the tool use and describe the tool
#   • A USER_QUESTION with a user query that should trigger the tool.
#       Example: "What is the weather in San Diego today?"

# Try experimenting with different system and user prompts
# ---------------------------------------------------------


# ---------------------------------------------------------
# Step 2: Create the prompt for the LLM to call tools
# ---------------------------------------------------------

USER_QUESTION = "What is the weather in San Diego today?"

# ---------------------------------------------------------
# Call the LLM with the prompts
# ---------------------------------------------------------
from openai import OpenAI
import json

client = OpenAI(api_key="ollama", base_url="http://localhost:11434/v1")

response = client.chat.completions.create(
    model="llama3.2:3b",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_QUESTION}
    ]
)

llm_output = response.choices[0].message.content
print("LLM Output:", llm_output)

# ---------------------------------------------------------
# Parse the LLM output and call the tool from Step 1
# ---------------------------------------------------------
if llm_output.startswith("TOOL_CALL:"):
    tool_call = json.loads(llm_output.replace("TOOL_CALL:", "").strip())
    if tool_call["name"] == "get_current_weather":
        city = tool_call["args"]["city"]
        unit = tool_call["args"].get("unit", "celsius")
        
        # Call the function from Step 1 (real API or dummy)
        tool_result = get_current_weather(city, unit)
        print("Tool Result:", tool_result)
    else:
               print("Unknown tool requested.")
else:
    print("No tool call detected in LLM output.")


LLM Output: TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Diego", "unit": "celsius"}}
Tool Result: It is 24.73°C and clear sky in San Diego.
Tool Result: It is 24.73°C and clear sky in San Diego.


Now that you have defined a tool and shown the model how to use it, the next step is to call the LLM using your prompt.

Start the **Ollama** server in a terminal with `ollama serve`. This launches a local API endpoint that listens for LLM requests. Once the server is running, return to the notebook and in the next cell send a query to the model.


In [46]:
from openai import OpenAI

client = OpenAI(api_key = "ollama", base_url = "http://localhost:11434/v1")

# ---------------------------------------------------------
# Step 3: Call the LLM with your prompt
# ---------------------------------------------------------
# Task:
#   Send SYSTEM_PROMPT + USER_QUESTION to the model.
#
# Steps:
#   1. Use the Ollama client to create a chat completion. 
#       - You may find some examples here: https://platform.openai.com/docs/api-reference/chat/create
#       - If you are unsure, search the web for "client.chat.completions.create"
#   2. Print the raw response.
#
# Expected:
#   The model should return something like:
#   TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Diego"}}
# ---------------------------------------------------------


# ---------------------------------------------------------
# Step 3: Call the LLM with your prompt
# ---------------------------------------------------------

# Create a chat completion using the Ollama-compatible OpenAI client
response = client.chat.completions.create(
    model="llama3.2:3b",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_QUESTION},
    ],
)

# Print the raw response object
print(response)

# Optionally, also print the assistant's message content for convenience
print("LLM Output:", response.choices[0].message.content)


ChatCompletion(id='chatcmpl-46', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Diego", "unit": "celsius"}}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1764106562, model='llama3.2:3b', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=30, prompt_tokens=179, total_tokens=209, completion_tokens_details=None, prompt_tokens_details=None))
LLM Output: TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Diego", "unit": "celsius"}}


In [47]:
# ---------------------------------------------------------
# Step 4: Parse the LLM output and call the tool
# ---------------------------------------------------------
# Task:
#   Detect when the model requests a tool, extract its name and arguments,
#   and execute the corresponding function.
#
# Steps:
#   1. Search for the text pattern "TOOL_CALL:{...}" in the model output.
#   2. Parse the JSON inside it to get the tool name and args.
#   3. Call the matching function (e.g., get_current_weather).
#
# Expected:
#   You should see a line like:
#       Calling tool `get_current_weather` with args {'city': 'San Diego'}
#       Result: It is 23°C and sunny in San Diego.
# ---------------------------------------------------------

import re, json


# ---------------------------------------------------------
# Step 4: Parse the LLM output and call the tool
# ---------------------------------------------------------

# Look for a TOOL_CALL followed by a JSON object
match = re.search(r"TOOL_CALL:\s*(\{.*\})", llm_output, re.DOTALL)
if match:
    tool_json_str = match.group(1)
    try:
        tool_call = json.loads(tool_json_str)
        name = tool_call.get("name")
        args = tool_call.get("args", {})
        print(f"Calling tool `{name}` with args {args}")

        if name == "get_current_weather":
            city = args.get("city")
            unit = args.get("unit", "celsius")
            result = get_current_weather(city, unit)
            print(f"Result: {result}")
        else:
            print(f"Unknown tool: {name}")
    except json.JSONDecodeError as e:
        print(f"Failed to parse TOOL_CALL JSON: {e}")
else:
    # If no tool call is found, output the model's response directly
    print("No TOOL_CALL found. LLM Response:")
    print(llm_output)


Calling tool `get_current_weather` with args {'city': 'San Diego', 'unit': 'celsius'}
Result: It is 24.73°C and clear sky in San Diego.


# 3- Standadize tool calling

So far, we handled tool calling manually by writing one regex and one hard-coded function. This approach does not scale if we want to add more tools. Adding more tools would mean more `if/else` blocks and manual edits to the `TOOL_SPEC` prompt.

To make the system flexible, we can standardize tool definitions by automatically reading each function’s signature, converting it to a JSON schema, and passing that schema to the LLM. This way, the LLM can dynamically understand which tools exist and how to call them without requiring manual updates to prompts or conditional logic.

Next, you will implement a small helper that extracts metadata from functions and builds a schema for each tool.

In [48]:
# ---------------------------------------------------------
# Generate a JSON schema for a tool automatically
# ---------------------------------------------------------
#
# Steps:
#   1. Use `inspect.signature` to get function parameters.
#   2. For each argument, record its name, type, and description.
#   3. Build a schema containing:
#   4. Test your helper on `get_current_weather` and print the result.
#
# Expected:
#   A dictionary describing the tool (its name, args, and types).
# ---------------------------------------------------------


# ---------------------------------------------------------
# Generate a JSON schema for a tool automatically
# ---------------------------------------------------------

from pprint import pprint
import inspect

def to_schema(fn):
    schema = {
        "name": fn.__name__,
        "description": fn.__doc__ or "No description available",
        "args": []
    }

    sig = inspect.signature(fn)
    for param_name, param in sig.parameters.items():
        arg_info = {
            "name": param_name,
            "type": str(param.annotation) if param.annotation != inspect._empty else "string",
            "default": param.default if param.default != inspect._empty else None
        }
        schema["args"].append(arg_info)

    return schema

# Test on# Test on get_current_weather
tool_schema = to_schema(get_current_weather)
pprint(tool_schema)


{'args': [{'default': None, 'name': 'city', 'type': "<class 'str'>"},
          {'default': 'celsius', 'name': 'unit', 'type': "<class 'str'>"}],
 'description': 'No description available',
 'name': 'get_current_weather'}


In [50]:
# ---------------------------------------------------------
# Generate a JSON schema for a tool automatically
# ---------------------------------------------------------
#
# Steps:
#   1. Use `inspect.signature` to get function parameters.
#   2. For each argument, record its name, type, and description.
#   3. Build a schema containing:
#   4. Test your helper on `get_current_weather` and print the result.
#
# Expected:
#   A dictionary describing the tool (its name, args, and types).
# ---------------------------------------------------------

from pprint import pprint
import inspect


def to_schema(fn):
    schema = {
        "name": fn.__name__,
        "description": fn.__doc__ or "No description available",
        "args": []
    }
    sig = inspect.signature(fn)
    for param_name, param in sig.parameters.items():
        arg_info = {
            "name": param_name,
            "type": str(param.annotation) if param.annotation != inspect._empty else "string",
            "default": param.default if param.default != inspect._empty else None
        }
        schema["args"].append(arg_info)

    return schema

tool_schema = to_schema(get_current_weather)
pprint(tool_schema)


{'args': [{'default': None, 'name': 'city', 'type': "<class 'str'>"},
          {'default': 'celsius', 'name': 'unit', 'type': "<class 'str'>"}],
 'description': 'No description available',
 'name': 'get_current_weather'}


In [51]:
# ---------------------------------------------------------
# Provide the tool schema to the model
# ---------------------------------------------------------
# Goal:
#   Give the model a "menu" of available tools so it can choose
#   which one to call based on the user’s question.
#
# Steps:
#   1. Add an extra system message (e.g., name="tool_spec")
#      containing the JSON schema(s) of your tools.
#   2. Include SYSTEM_PROMPT and the user question as before.
#   3. Send the messages to the model (e.g., llama3.2:3b).
#   4. Print the raw model output to see if it picks the right tool.
#
# Expected:
#   The model should produce a structured TOOL_CALL indicating
#   which tool to use and with what arguments.
# ---------------------------------------------------------


# ---------------------------------------------------------
# Provide the tool schema to the model
# ---------------------------------------------------------

# Build the tool schema using the helper from previous step
tool_schema = to_schema(get_current_weather)

# Create messages for the LLM
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "system", "name": "tool_spec", "content": json.dumps(tool_schema)},
    {"role": "user", "content": USER_QUESTION}
]

# Send the messages to the model
response = client.chat.completions.create(
    model="llama3.2:3b",
    messages=messages
)

# Print raw response and the model's output
print("Raw Response:", response)
print("LLM Output:", response.choices[0].message.content)

Raw Response: ChatCompletion(id='chatcmpl-969', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Diego", "unit": "celsius"}}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1764106601, model='llama3.2:3b', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=30, prompt_tokens=241, total_tokens=271, completion_tokens_details=None, prompt_tokens_details=None))
LLM Output: TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Diego", "unit": "celsius"}}


## 4- LangChain for Tool Calling
So far, you built a simple tool-calling pipeline manually. While this helps you understand the logic, it does not scale well when working with multiple tools, complex parsing, or multi-step reasoning.

LangChain simplifies this process. You only need to declare your tools, and its *Agent* abstraction handles when to call a tool, how to use it, and how to continue reasoning afterward.

In this section, you will use the **ReAct** Agent (Reasoning + Acting). It alternates between reasoning steps and tool use, producing clearer and more reliable results. We will explore reasoning-focused models in more depth next week.

The following links might be helpful:
- https://python.langchain.com/api_reference/langchain/agents/langchain.agents.initialize.initialize_agent.html
- https://python.langchain.com/docs/integrations/tools/
- https://python.langchain.com/docs/integrations/chat/ollama/
- https://python.langchain.com/api_reference/core/language_models/langchain_core.language_models.llms.LLM.html

In [83]:
# ---------------------------------------------------------
# Step 1: Define tools for LangChain
# ---------------------------------------------------------
# Goal:
#   Convert your weather function into a LangChain-compatible tool.
#
# Steps:
#   1. Import `tool` from `langchain.tools`.
#   2. Keep your existing `get_current_weather` helper as before.
#   3. Create a new function (e.g., get_weather) that calls it.
#   4. Add the `@tool` decorator so LangChain can register it automatically.
#
# Notes:
#   • The decorator converts your Python function into a standardized tool object.
#   • Start with keeping the logic simple and offline-friendly.

from langchain.tools import tool


@tool
def get_weather(city: str) -> str:
    """Get the current weather for a city. Returns temperature in Celsius and weather conditions.
    
    Args:
        city: The name of the city to get weather for (e.g., 'Seattle', 'San Francisco')
    
    Returns:
        A string describing the current weather, including temperature and conditions.
    """
    # Strip any quotes that the LLM might have included
    city = city.strip().strip("'\"")
    
    # Add country code for US cities to improve API lookup
    # OpenWeatherMap API works better with "City,CountryCode" format
    if "," not in city:
        # Assume US cities for common names
        city_with_country = f"{city},US"
    else:
        city_with_country = city
    
    # Call the real API function with proper formatting
    result = get_current_weather(city_with_country, unit="celsius")
    
    # If we got an error with the country code, try without it
    if "Could not retrieve weather" in result and city_with_country != city:
        result = get_current_weather(city, unit="celsius")
    
    return result


In [84]:
# ---------------------------------------------------------
# Step 2: Initialize the LangChain Agent
# ---------------------------------------------------------
# Goal:
#   Connect your tool to a local LLM using LangChain's ReAct-style agent.
#
# Steps:
#   1. Import the required classes:
#        - ChatOllama (for local model access)
#        - initialize_agent, Tool, AgentType
#   2. Create an LLM instance (e.g., model="llama3.2:3b", temperature=0).
#   3. Add your tool(s) to a list
#   4. Initialize the agent using initialize_agent
#   5. Test the agent with a natural question (e.g., "Do I need an umbrella in Seattle today?").
#
# Expected:
#   The model should reason through the question, call your tool,
#   and produce a final answer in plain language.
# ---------------------------------------------------------

from langchain_community.chat_models import ChatOllama
from langchain.agents import initialize_agent, Tool, AgentType


# ---------------------------------------------------------
# Step 2: Initialize the LangChain Agent
# ---------------------------------------------------------

print("Using llama3.2:3b - a larger model with better ReAct capabilities")
print("Make sure you've pulled this model with: ollama pull llama3.2:3b\n")

# 1. Create an LLM instance using ChatOllama with a larger model
llm = ChatOllama(model="llama3.2:3b", temperature=0)

# 2. Add your tool(s) to a list
tools = [get_weather]  # This is the LangChain-compatible tool from Step 1

# 3. Initialize the agent using ReAct style with error handling
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3,  # Reduced to catch issues faster
    early_stopping_method="generate"  # Better handling when max iterations reached
)

# 4. Test the agent with a natural question
try:
    result = agent.run("Do I need an umbrella in Seattle today?")
    print("\n" + "="*50)
    print("Final Result:")
    print(result)  # agent.run() returns a string, not a dict
    
except Exception as e:
    print(f"\nError with ReAct agent: {e}")
    print("\nTroubleshooting:")
    print("1. Make sure llama3.2:3b is installed: ollama pull llama3.2:3b")
    print("2. Make sure Ollama server is running: ollama serve")
    print("3. Check that the model has enough memory to run")


Using llama3.2:3b - a larger model with better ReAct capabilities
Make sure you've pulled this model with: ollama pull llama3.2:3b



> Entering new AgentExecutor chain...
Question: Do I need an umbrella in Seattle today?

Thought: To determine if I need an umbrella, I should get the current weather for Seattle.

Action: get_weather
Action Input: 'Seattle'
Observation: It is 6.36°C and light rain in Seattle,US.
Thought:Question: Do I need an umbrella in Seattle today?

Thought: To determine if I need an umbrella, I should get the current weather for Seattle.

Action: get_weather
Action Input: 'Seattle'
Observation: It is 6.36°C and light rain in Seattle,US.
Thought:Question: Do I need an umbrella in Seattle today?

Thought: To determine if I need an umbrella, I should check the weather conditions. Since it's light rain, I probably don't need an umbrella.

Action: get_weather
Action Input: 'Seattle'
Observation: It is 6.36°C and light rain in Seattle,US.
Thought:Question: Do I need an u

### What just happened?

The console log displays the **Thought → Action → Observation → …** loop until the agent produces its final answer. Because `verbose=True`, LangChain prints each intermediate reasoning step.

If you want to add more tools, simply append them to the tools list. LangChain will handle argument validation, schema generation, and tool-calling logic automatically.


## 5- Perplexity‑Style Web Search
Agents become much more powerful when they can look up real information on the web instead of relying only on their internal knowledge.

In this section, you will combine everything you have learned to build a simple Ask-the-Web Agent. You will integrate a web search tool (DuckDuckGo) and make it available to the agent using the same tool-calling approach as before.

This will let the model retrieve fresh results, reason over them, and generate an informed answer—similar to how Perplexity works.

You may find some examples from the following links:
- https://pypi.org/project/duckduckgo-search/

In [94]:
# ---------------------------------------------------------
# Step 1: Add a web search tool
# ---------------------------------------------------------
# Goal:
#   Create a tool that lets the agent search the web and return results.
#
# Steps:
#   1. Use DuckDuckGo for quick, open web searches.
#   2. Write a helper function (e.g., search_web) that:
#        • Takes a query string
#        • Uses DDGS to fetch top results (titles + URLs)
#        • Returns them as a formatted string
#   3. Wrap it with the @tool decorator to make it available to LangChain.


from ddgs import DDGS
from langchain.tools import tool

@tool
def web_search(query: str) -> str:
    """Search the web for information using DuckDuckGo. Returns top search results with titles and URLs.
    
    Args:
        query: The search query string
    
    Returns:
        A formatted string containing search results with titles and URLs.
    """
    # Strip any quotes that the LLM might have included
    query = query.strip().strip("'\"")
    print(query)
    
    try:
        # Use DDGS to search the web
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=5))
        
        # Format the results into a readable string
        if not results:
            return f"No search results found for: {query}"
        
        formatted_results = f"Search results for '{query}':\n\n"
        for i, result in enumerate(results, 1):
            title = result.get('title', 'No title')
            url = result.get('href', 'No URL')
            snippet = result.get('body', 'No description')
            formatted_results += f"{i}. {title}\n   URL: {url}\n   {snippet}\n\n"
        
        return formatted_results
    
    except Exception as e:
        return f"Error searching the web: {str(e)}"


In [96]:

# ---------------------------------------------------------
# Step 2: Initialize the web-search agent
# ---------------------------------------------------------
# Goal:
#   Connect your `web_search` tool to a language model
#   so the agent can search and reason over real data.
#
# Steps:
#   1. Import `initialize_agent` and `AgentType`.
#   2. Create an LLM (e.g., ChatOllama).
#   3. Add your `web_search` tool to the tools list.
#   4. Initialize the agent using: initialize_agent
#   5. Keep `verbose=True` to observe reasoning steps.
#
# Expected:
#   The agent should be ready to accept user queries
#   and use your web search tool when needed.
# ---------------------------------------------------------
from langchain.agents import initialize_agent, AgentType
from langchain_community.chat_models import ChatOllama

# Create an LLM instance using ChatOllama
llm = ChatOllama(model="llama3.2:3b", temperature=0)

# Add the web_search tool to the tools list
web_tools = [web_search]

# Initialize the agent with the web search tool
web_agent = initialize_agent(
    tools=web_tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)


Temperature controls the randomness of the LLM's responses:

Temperature = 0: Makes the model completely deterministic - it always picks the most likely next token. This gives consistent, predictable outputs.
Higher temperature (e.g., 0.7-1.0): Increases randomness and creativity, making responses more varied but potentially less focused.
For tool-calling agents, temperature=0 is preferred because:

Consistent reasoning: The ReAct agent needs to reliably follow the Thought → Action → Observation pattern
Reliable tool calls: You want the agent to consistently format tool calls correctly (JSON syntax matters)
Predictable behavior: When dealing with real APIs and external tools, deterministic behavior makes debugging easier
Factual accuracy: Lower temperature reduces hallucination risk when the agent needs to be precise
You could increase the temperature (e.g., to 0.3-0.5) if you want more creative responses in the final answer, but for structured tasks like tool calling, keeping it at 0 is the standard practice.


Let’s see the agent's output in action with a real example.


In [99]:

# ---------------------------------------------------------
# Step 3: Test your Ask-the-Web agent
# ---------------------------------------------------------
# Goal:
#   Verify that the agent can search the web and return
#   a summarized answer based on real results.
#
# Steps:
#   1. Ask a natural question that requires live information,
#      for example: "What are the current events in San Francisco this week?"
#   2. Call agent.
#
# Expected:
#   The agent should call `web_search`, retrieve results,
#   and generate a short summary response.
# ---------------------------------------------------------

try:
    # Ask a question that requires current web information
    result = web_agent.run("What are the current events in San Francisco this week?")
    
    print("\n" + "="*50)
    print("Final Result:")
    print(result)
    
except Exception as e:
    print(f"\nError running web agent: {e}")
    print("\nTroubleshooting:")
    print("1. Make sure Ollama server is running: ollama serve")
    print("2. Verify duckduckgo-search is installed: pip install duckduckgo-search")
    print("3. Check your internet connection")





> Entering new AgentExecutor chain...
Thought: To find the current events in San Francisco for this week, I should use the web_search tool.

Action: web_search
Action Input: "current events in San Francisco this week"current events in San Francisco this week
Thought: To find the current events in San Francisco for this week, I should use the web_search tool.

Action: web_search
Action Input: "current events in San Francisco this week"current events in San Francisco this week

Observation: Search results for 'current events in San Francisco this week':

1. ABC7 News - KGO Bay Area and San Francisco News
   URL: 
   No one covers SanFrancisco weather and the surrounding Bay Area like ABC7. KGO covers forecasts, weather maps, alerts, video, street-level weather and more.Watch live streaming video on ABC7news.com and stay up-to-date with the latest KGO news broadcasts as well as live breaking news whenever it happens.ABC7 covers local SanFrancisco news, weather, sports and traffic so you


## 6- A minimal UI
This project includes a simple **React** front end that sends the user’s question to a FastAPI back end and streams the agent’s response in real time. To run the UI:

1- Open a terminal and start the Ollama server: `ollama serve`.

2- In a second terminal, navigate to the frontend folder and install dependencies:`npm install`.

3- In the same terminal, navigate to the backend folder and start the FastAPI back‑end: `uvicorn app:app --reload --port 8000`

4- Open a third terminal, navigate to the frontend folder, and start the React dev server: `npm run dev`

5- Visit `http://localhost:5173/` in your browser.



## 🎉 Congratulations!

* You have built a **web‑enabled agent**: tool calling → JSON schema → LangChain ReAct → web search → simple UI.
* Try adding more tools, such as news or finance APIs.
* Experiment with multiple tools, different models, and measure accuracy vs. hallucination.


👏 **Great job!** Take a moment to celebrate. The techniques you implemented here power many production agents and chatbots.